In [31]:
import json
from geopandas import GeoDataFrame
from shapely.geometry import Polygon, Point, LineString
import geopandas as gpd
import pandas as pd
from datetime import datetime
import re

import folium
from folium import PolyLine
from folium.plugins import MarkerCluster

### Folium Map

Читаю датасет с координатами событий и конвертирую его в json, предварительно объявив его GeoDataFram'ом. С ним намного удобнее работать :)

In [42]:
event_coords = pd.read_csv('event_coo.csv')
geometry = [Point(xy) for xy in zip(event_coords.place_lon, event_coords.place_lat)]
event_coords = GeoDataFrame(event_coords, geometry=geometry)
json_coords = json.loads(event_coords.to_json())['features']

Читаю нужные колонки из сета по событиям

In [43]:
event = pd.read_csv('D:/event.csv', usecols = ['city', 'begin', 'id', 'name', 'place', 'status',])
event.head(2)

,begin,city,id,name,place,status
0,2013-04-03 12:00:00,Каменец-Подольский,61435,"Деловая игра ""Железный предприниматель"" Камене...",Место и адрес проведения уточняются.,ok
1,2013-04-06 15:00:00,Москва,61436,"Лекция Елизаветы Муратовой «Фотокомпозиция, цв...","ул. Большая Полянка, 65/74, стр. 1. FAQcafe",ok


Выбираем Москву (как пример) и мерджим с координатами событий

In [44]:
event_moscow = event[event.city=='Москва'] # 
event_moscow = pd.merge(event_moscow, event_coords, how='left', left_on='id', right_on='event_id')

In [45]:
#event_moscow = event_moscow.set_index('id')

In [46]:
event_moscow.head()

,begin,city,id,name,place,status,event_id,place_lat,place_lon,subw_lat,subw_lon,geometry
0,2013-04-06 15:00:00,Москва,61436,"Лекция Елизаветы Муратовой «Фотокомпозиция, цв...","ул. Большая Полянка, 65/74, стр. 1. FAQcafe",ok,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-03-29 12:00:00,Москва,61439,Встреча с пайщиками,"ул.Сущевская, д.21, подъезд 2, 3 этаж, офис 2,...",ok,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-04-03 12:00:00,Москва,61440,Встреча с пайщиками,"ул.Сущевская, д.21, подъезд 2, 3 этаж, офис 2,...",ok,61440.0,55.78231,37.600494,55.781788,37.598736,POINT (37.60049438476563 55.78231048583984)
3,2013-04-06 14:00:00,Москва,61444,Сказка о потерянном времени!,Старый Арбат,ok,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-04-08 19:00:00,Москва,61453,Игорь Оболенский. Украсть Пиросмани,"ул. Дружинниковская, 11/2",ok,61453.0,55.75753,37.575027,55.760212,37.577209,POINT (37.57502746582031 55.75753021240234)


In [47]:
cols = ['id'
        ,'city'
        ,'begin'
        ,'name'
        ,'place'
        ,'place_lat'
        ,'place_lon'
        ,'geometry'
        ]

event_moscow = event_moscow[cols]
#json_event_moscow = GeoDataFrame(event_moscow[cols].to_json())

Дропнем все значения, где не были записаны координаты, такие точки нас не интересуют

In [48]:
event_moscow = event_moscow.dropna(subset=['geometry'], how='all')
event_moscow.head(2)

,id,city,begin,name,place,place_lat,place_lon,geometry
2,61440,Москва,2013-04-03 12:00:00,Встреча с пайщиками,"ул.Сущевская, д.21, подъезд 2, 3 этаж, офис 2,...",55.78231,37.600494,POINT (37.60049438476563 55.78231048583984)
4,61453,Москва,2013-04-08 19:00:00,Игорь Оболенский. Украсть Пиросмани,"ул. Дружинниковская, 11/2",55.75753,37.575027,POINT (37.57502746582031 55.75753021240234)


У json'a созданного выше есть свои id, а внутри блоков запрятан event_id. Создадим словарь event_id -> json_id. Пригодится при рисовании карты

In [49]:
json_ids = []
event_ids = []
for id in range(len(json_coords)):
    json_ids.append(int(json_coords[id]['id']))
    event_ids.append(int(json_coords[id]['properties']['event_id']))
id_map_dict = dict(zip(event_ids,json_ids))

Создаем карту с центром в центра Москвы, выбираем 999 событий и делаем + строим их кластеризацию. 
Всплывающее окно представляет собой html вставку и по-хорошему надо все значки типа кавычек и др. заключать в соответствующие теги. Для просто ты я просто убрал все символы кроме букв.

Карта готова. :)

In [50]:
m = folium.Map(location=[55.764414, 37.647859], zoom_start=10)
marker_cluster = MarkerCluster().add_to(m)
num_events = 1000
for event_id in list(event_moscow.id.values[:num_events]):

    name = re.sub('\W+',' ',event_moscow[event_moscow.id==event_id].name.values[0])
    try:
        place = re.sub('\W+',' ',event_moscow[event_moscow.id==event_id].place.values[0])
    except:
        place = '-'
    city = event_moscow[event_moscow.id==event_id].city.values[0]
    date = event_moscow[event_moscow.id==event_id].begin.values[0]
    latitude = event_moscow[event_moscow.id==event_id].place_lon.values[0]
    longitude = event_moscow[event_moscow.id==event_id].place_lat.values[0]
    
    #print(city, place, date, name, latitude, longitude)
    event_description = folium.Html("""
    <b>Город</b>: {} 
    <br><b>Адрес</b>: {} 
    <br><b>Дата</b>: {} 
    <br><br><b>Описание</b>: {} 
    <br><br><b>Широта</b>: {} 
    <br><b>Долгота</b>: {} """.format(city, place, date, name, latitude, longitude), script=True)
    
    popup = folium.Popup(event_description, max_width=2650)
    json_id = id_map_dict[event_id]
    folium.Marker(list(reversed(json_coords[json_id]['geometry']['coordinates'])), 
                        popup=popup, 
                 #       radius=1, 
                 #       color='#000000'
                 ).add_to(marker_cluster)
    
for moscow_region in moscow_json:
    locations = moscow_region['geometry']['coordinates']
    adm_unit = moscow_region['properties']['OKRUGS_RUS']
    for subregion in locations:
        if len(locations)==1: # Single polygon
            folium.PolyLine(
            locations=[list(reversed(x)) for x in subregion],
            color=dict_colors[adm_unit],
            weight=1,
            fill_color=dict_colors[adm_unit],
            fill_opacity=0.2,
            fill=True,
            popup=moscow_region['properties']['RAION_RUS'],
        ).add_to(m)
        else: # Multi polygons
            folium.PolyLine(
            locations=[list(reversed(x)) for x in subregion[0]],
            color=dict_colors[adm_unit],
            weight=1,
            fill_color=dict_colors[adm_unit],
            fill_opacity=0.2,
            fill=True,
            popup=moscow_region['properties']['RAION_RUS'],
        ).add_to(m)

m.save('Moscow Events Map Top-'+str(num_events)+'.html')
m

NameError: name 'moscow_json' is not defined

### Moscow Regions

In [8]:
import shapefile
from json import dumps

In [9]:
path = "./maps/administrative-divisions-of-moscow/moscow_adm.shp"
moscow_shapefile = gpd.read_file(path)
moscow_shapefile.head()

,OKATO,OKTMO,RAION,OKRUGS,geometry
0,45298555,45945000,Poselenie Kievskij,Troitsky,"(POLYGON ((36.8031012 55.4408329, 36.8031903 5..."
1,45268595,45328000,Filevskij Park,Western,"POLYGON ((37.4276499 55.7482092, 37.4284863 55..."
2,45298567,45954000,Poselenie Novofedorovskoe,Troitsky,"POLYGON ((36.8035692 55.4516224, 36.8045117 55..."
3,45298575,45956000,Poselenie Rogovskoe,Troitsky,"POLYGON ((36.9372397 55.2413907, 36.9372604 55..."
4,45297568,45953000,Poselenie Mosrentgen,Novomoskovsky,"POLYGON ((37.4395575 55.6273129, 37.4401803 55..."


In [10]:
dict_mokrugs_eng_rus

NameError: name 'dict_mokrugs_eng_rus' is not defined

In [11]:
moscow_okrugs_list = ['Новомосковский', 'Юго-Западный', 'Северо-Восточный', 'Центральный', 'Троицкий',
                      'Восточный', 'Юго-Восточный', 'Зеленоградский', 'Южный', 'Северный',  'Северо-Западный', 'Западный']

#print(set(moscow_shapefile.OKRUGS))

dict_mokrugs_eng_rus = dict(zip(set(moscow_shapefile.OKRUGS), moscow_okrugs_list))
dict_mokrugs_eng_rus

{'Central': 'Центральный',
 'Eastern': 'Новомосковский',
 'North-Eastern': 'Западный',
 'North-Western': 'Южный',
 'Northern': 'Зеленоградский',
 'Novomoskovsky': 'Восточный',
 'South-Eastern': 'Юго-Западный',
 'South-Western': 'Северо-Западный',
 'Southern': 'Северный',
 'Troitsky': 'Троицкий',
 'Western': 'Юго-Восточный',
 'Zelenogradsky': 'Северо-Восточный'}

In [12]:
moscow_shapefile['OKRUGS_RUS'] = moscow_shapefile['OKRUGS'].map(lambda x: dict_mokrugs_eng_rus[x])
moscow_shapefile['RAION_RUS'] = moscow_shapefile['RAION'].map(lambda x: dict_mregions_eng_rus[x])
moscow_shapefile.head()

NameError: name 'dict_mregions_eng_rus' is not defined

In [19]:
moscow_json = json.loads(GeoDataFrame(moscow_shapefile).to_json())['features']

In [20]:
moscow_json[0]

{'geometry': {'coordinates': [[[[36.8031012, 55.4408329],
     [36.8031903, 55.4416007],
     [36.8035692, 55.4516224],
     [36.812528, 55.4513994],
     [36.8274471, 55.4513398],
     [36.8333688, 55.4513764],
     [36.8338034, 55.4516439],
     [36.8345763, 55.4512558],
     [36.8348594, 55.4514247],
     [36.8349932, 55.4514931],
     [36.8358013, 55.4511173],
     [36.8360591, 55.4511632],
     [36.8461554, 55.4510412],
     [36.8602864, 55.4508946],
     [36.8649423, 55.4506415],
     [36.8608407, 55.4492656],
     [36.8582649, 55.4478456],
     [36.8582898, 55.447659],
     [36.8600008, 55.4466656],
     [36.8611076, 55.4473042],
     [36.8622805, 55.4467128],
     [36.8638768, 55.4472018],
     [36.8694408, 55.4489425],
     [36.8724625, 55.4502245],
     [36.8749845, 55.4513839],
     [36.8773319, 55.453135],
     [36.8804877, 55.4548177],
     [36.8822676, 55.455771],
     [36.8833225, 55.4551478],
     [36.8837761, 55.4554817],
     [36.8846681, 55.4551548],
     [36.8855977

In [14]:
def adm_unit_color(adm_unit):
    if adm_unit == 'Новомосковский':
        color = '#e6194b'
    elif adm_unit == 'Юго-Западный':
        color = '#3cb44b'
    elif adm_unit == 'Северо-Восточный':
        color = '#ffe119'
    elif adm_unit == 'Центральный':
        color = '#0082c8'
    elif adm_unit == 'Троицкий':
        color = '#f58231'
    elif adm_unit == 'Восточный':
        color = '#911eb4'
    elif adm_unit == 'Юго-Восточный':
        color = '#000080'
    elif adm_unit == 'Зеленоградский':
        color = '#f032e6'
    elif adm_unit == 'Южный':
        color = '#d2f53c'
    elif adm_unit == 'Северный':
        color = '#fabebe'
    elif adm_unit == 'Северо-Западный':
        color = '#008080'
    elif adm_unit == 'Западный':
        color = '#800000'
    return color

In [15]:
colors = []
for adm_unit in set(moscow_shapefile['OKRUGS_RUS']):
    colors.append(adm_unit_color(adm_unit))
    
colors

['#fabebe',
 '#d2f53c',
 '#3cb44b',
 '#f58231',
 '#ffe119',
 '#008080',
 '#f032e6',
 '#0082c8',
 '#800000',
 '#e6194b',
 '#911eb4',
 '#000080']

In [16]:
dict_colors = dict(zip( set(moscow_shapefile['OKRUGS_RUS']), colors))

In [17]:
m = folium.Map(location=[55.764414, 37.647859], zoom_start=10)

for moscow_region in moscow_json:
    locations = moscow_region['geometry']['coordinates']
    adm_unit = moscow_region['properties']['OKRUGS_RUS']
    for subregion in locations:
        if len(locations)==1: # Single polygon
            folium.PolyLine(
            locations=[list(reversed(x)) for x in subregion],
            color=dict_colors[adm_unit],
            weight=1,
            fill_color=dict_colors[adm_unit],
            fill_opacity=0.2,
            fill=True,
            popup=moscow_region['properties']['RAION_RUS'],
        ).add_to(m)
        else: # Multi polygons
            folium.PolyLine(
            locations=[list(reversed(x)) for x in subregion[0]],
            color=dict_colors[adm_unit],
            weight=1,
            fill_color=dict_colors[adm_unit],
            fill_opacity=0.2,
            fill=True,
            popup=moscow_region['properties']['RAION_RUS'],
        ).add_to(m)
    
m

NameError: name 'moscow_json' is not defined

In [105]:
moscow_regions_stats = pd.read_csv('moscow_regions_stats1.csv', encoding="utf-8", sep = ';')
dict_mregions_eng_rus = dict(zip(moscow_regions_stats.eng_region_name.values, moscow_regions_stats.rus_region_name.values))

moscow_regions_stats.head(2)

,eng_region_name,rus_region_name,adm_unit,area,population_2017,density_pkm
0,Akademicheskoe,Академическое,ЮЗАО,5.83,109127,18718.18
1,Alekseevskoe,Алексеевское,СВАО,5.29,80339,15186.96


In [34]:
# read the shapefile
reader = shapefile.Reader("./maps/administrative-divisions-of-moscow/moscow_adm.shp")
fields = reader.fields[1:]
field_names = [field[0] for field in fields]
buffer = []
for sr in reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    geom = sr.shape.__geo_interface__
    buffer.append(dict(type="Feature", geometry=geom, properties=atr)) 
    
# write the GeoJSON file
    
geojson = open("moscow_city.json", "w")
geojson.write(dumps({"type": "FeatureCollection","features": buffer}, indent=2) + "\n")
geojson.close()

In [ ]:
json_coords = json.loads(event_coords.to_json())['features']

In [98]:

from matplotlib import colors

In [107]:
set()

{'Central',
 'Eastern',
 'North-Eastern',
 'North-Western',
 'Northern',
 'Novomoskovsky',
 'South-Eastern',
 'South-Western',
 'Southern',
 'Troitsky',
 'Western',
 'Zelenogradsky'}

In [93]:
json_test = json.loads(moscow_regions_stats.to_json())

In [66]:
regions = []
for region in moscow_json:
    regions.append(region['properties']['RAION'])

In [68]:
set(regions)

{'Ajeroport',
 'Akademicheskoe',
 'Alekseevskoe',
 "Altuf'evskoe",
 'Arbat',
 'Babushkinskoe',
 'Basmannoe',
 'Begovoe',
 'Beskudnikovskoe',
 'Bibirevo',
 'Birjulevo Vostochnoe',
 'Birjulevo Zapadnoe',
 'Bogorodskoe',
 'Brateevo',
 'Butyrskoe',
 'Caricyno',
 'Cheremushki',
 "Chertanovo Central'noe",
 'Chertanovo Juzhnoe',
 'Chertanovo Severnoe',
 'Danilovskoe',
 'Dmitrovskoe',
 'Donskoe',
 'Dorogomilovo',
 'Filevskij Park',
 'Fili Davydkovo',
 'Gagarinskoe',
 "Gol'janovo",
 'Golovinskoe',
 'Hamovniki',
 'Horoshevo-Mnevniki',
 'Horoshevskoe',
 'Hovrino',
 'Ivanovskoe',
 'Izmajlovo',
 'Jakimanka',
 'Jaroslavskoe',
 'Jasenevo',
 'Juzhnoe Butovo',
 'Juzhnoe Medvedkovo',
 'Juzhnoe Tushino',
 'Juzhnoportovoe',
 'Kapotnja',
 "Kon'kovo",
 'Koptevo',
 'Kosino-Uhtomskoe',
 'Kotlovka',
 "Krasnosel'skoe",
 'Krjukovo',
 'Krylatskoe',
 'Kuncevo',
 'Kurkino',
 "Kuz'minki",
 'Lefortovo',
 'Levoberezhnoe',
 'Lianozovo',
 'Ljublino',
 'Lomonosovskoe',
 'Losinoostrovskoe',
 "Mar'ina Roshha",
 "Mar'ino",


### LayerControl + FeatureGroup

In [182]:
from folium import FeatureGroup, LayerControl, Map, Marker

In [174]:
event_moscow.head()

,begin,city,id,name,place,status,event_id,place_lat,place_lon,subw_lat,subw_lon,geometry
2,2013-04-03 12:00:00,Москва,61440,Встреча с пайщиками,"ул.Сущевская, д.21, подъезд 2, 3 этаж, офис 2,...",ok,61440.0,55.782310,37.600494,55.781788,37.598736,POINT (37.60049438476563 55.78231048583984)
4,2013-04-08 19:00:00,Москва,61453,Игорь Оболенский. Украсть Пиросмани,"ул. Дружинниковская, 11/2",ok,61453.0,55.757530,37.575027,55.760212,37.577209,POINT (37.57502746582031 55.75753021240234)
6,2013-03-27 19:00:00,Москва,61456,К 400-летию Дома Романовых: документальное кин...,"ул.Каретный ряд, д.5/10",ok,61456.0,55.772678,37.608418,55.765842,37.608166,POINT (37.60841751098633 55.77267837524414)
7,2013-04-13 10:00:00,Москва,61457,Деловая игра «Управление регионом: почувствуй ...,"пр.Вернадского, 82 с3",ok,61457.0,55.668808,37.476753,55.663147,37.482853,POINT (37.47675323486328 55.66880798339844)
12,2013-03-30 17:30:00,Москва,61473,"""Ангелы и многое другое"" в цикле ""Просто батик...",м.Чистые пруды,ok,61473.0,55.764793,37.638683,55.764793,37.638683,POINT (37.6386833190918 55.7647933959961)


In [177]:
print(type(event_moscow.begin.values[0]))

<class 'str'>


In [178]:
event_moscow.shape

(236659, 12)

In [179]:
event_moscow['year'] = event_moscow['begin'].map(lambda x: x[:4])

In [180]:
event_moscow.head()

,begin,city,id,name,place,status,event_id,place_lat,place_lon,subw_lat,subw_lon,geometry,year
2,2013-04-03 12:00:00,Москва,61440,Встреча с пайщиками,"ул.Сущевская, д.21, подъезд 2, 3 этаж, офис 2,...",ok,61440.0,55.782310,37.600494,55.781788,37.598736,POINT (37.60049438476563 55.78231048583984),2013
4,2013-04-08 19:00:00,Москва,61453,Игорь Оболенский. Украсть Пиросмани,"ул. Дружинниковская, 11/2",ok,61453.0,55.757530,37.575027,55.760212,37.577209,POINT (37.57502746582031 55.75753021240234),2013
6,2013-03-27 19:00:00,Москва,61456,К 400-летию Дома Романовых: документальное кин...,"ул.Каретный ряд, д.5/10",ok,61456.0,55.772678,37.608418,55.765842,37.608166,POINT (37.60841751098633 55.77267837524414),2013
7,2013-04-13 10:00:00,Москва,61457,Деловая игра «Управление регионом: почувствуй ...,"пр.Вернадского, 82 с3",ok,61457.0,55.668808,37.476753,55.663147,37.482853,POINT (37.47675323486328 55.66880798339844),2013
12,2013-03-30 17:30:00,Москва,61473,"""Ангелы и многое другое"" в цикле ""Просто батик...",м.Чистые пруды,ok,61473.0,55.764793,37.638683,55.764793,37.638683,POINT (37.6386833190918 55.7647933959961),2013


In [190]:
map = Map(
    location=[45.372, -121.6972],
    zoom_start=12,
    tiles='Stamen Terrain'
)


In [191]:
feature_group = FeatureGroup(name='One icon')
Marker(location=[45.3288, -121.6625],
       popup='Mt. Hood Meadows').add_to(feature_group)

feature_group1 = FeatureGroup(name='Another icon')
Marker(location=[45.3311, -121.7113],
       popup='Timberline Lodge').add_to(feature_group1)

feature_group.add_to(map)
feature_group1.add_to(map)
LayerControl().add_to(map)
map

In [2]:
import folium
from folium.plugins import TimestampedGeoJson
import numpy as np

coordinates = [[[[lon-8*np.sin(theta), -47+6*np.cos(theta)] for
                 theta in np.linspace(0, 2*np.pi, 25)],
                [[lon-4*np.sin(theta), -47+3*np.cos(theta)] for theta
                 in np.linspace(0, 2*np.pi, 25)]] for
               lon in np.linspace(-150, 150, 7)]
data = {
    "type": "FeatureCollection",
    "features": [
            {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [0, 0],
                    },
                "properties": {
                    "times": [1435708800000+12*86400000]
                    }
                },
        {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [-50, 50],
                    },
                "properties": {
                    "times": [1435708800000]
                    }
                },
            {
                "type": "Feature",
                "geometry": {
                    "type": "MultiPoint",
                    "coordinates": [[lon, -25] for
                                    lon in np.linspace(-150, 150, 49)],
                    },
                "properties": {
                    "times": [1435708800000+i*86400000 for
                              i in np.linspace(0, 25, 49)]
                    }
                },
            {
              "geometry": {
                "coordinates": [
                  50.0,
                  50.0
                ],
                "type": "Point"
              },
              "id": "1",
              "properties": {"foo": "bar"},
              "type": "Feature",
              "when": {
                "start": "2016-04-24",
                "end": "2016-04-26",
                "type": "Interval"
              }
    }
        ],
    }

m = folium.Map([47, 3], zoom_start=1)
tgj = TimestampedGeoJson(data, period='P1D')
m.add_child(tgj)
m
del m.template_vars.get('custom_markers')[0]

AttributeError: 'Map' object has no attribute 'template_vars'

In [113]:
def generate_json(first_date):
    len = 10
    import random
    d = {}
    lst = [[x*10,y*10] for x,y in zip(range(len),range(len))]
    d_cont = {}
    d_times = {}
    time_list = [1435708800000+i*86400000 for i in range(len)]
    d_times['times'] = time_list
    d_cont['coordinates'] = lst
    d_cont['type'] = 'MultiPoint'
    d['geometry'] = d_cont
    d['properties'] = d_times
    d['type'] = 'Feature'
    return d

In [114]:
generate_json(1)

{'geometry': {'coordinates': [[0, 0],
   [10, 10],
   [20, 20],
   [30, 30],
   [40, 40],
   [50, 50],
   [60, 60],
   [70, 70],
   [80, 80],
   [90, 90]],
  'type': 'MultiPoint'},
 'properties': {'times': [1435708800000,
   1435795200000,
   1435881600000,
   1435968000000,
   1436054400000,
   1436140800000,
   1436227200000,
   1436313600000,
   1436400000000,
   1436486400000]},
 'type': 'Feature'}

In [115]:
data1 = generate_json(1)
m = folium.Map([47, 3], zoom_start=1)
tgj = TimestampedGeoJson(data1, period='P1D')
m.add_child(tgj)
m

In [108]:
import random
random.randint(0,100)

79

In [25]:
data['features']

[{'geometry': {'coordinates': [0, 0], 'type': 'Point'},
  'properties': {'times': [1436745600000]},
  'type': 'Feature'},
 {'geometry': {'coordinates': [[-150.0, -25],
    [-143.75, -25],
    [-137.5, -25],
    [-131.25, -25],
    [-125.0, -25],
    [-118.75, -25],
    [-112.5, -25],
    [-106.25, -25],
    [-100.0, -25],
    [-93.75, -25],
    [-87.5, -25],
    [-81.25, -25],
    [-75.0, -25],
    [-68.75, -25],
    [-62.5, -25],
    [-56.25, -25],
    [-50.0, -25],
    [-43.75, -25],
    [-37.5, -25],
    [-31.25, -25],
    [-25.0, -25],
    [-18.75, -25],
    [-12.5, -25],
    [-6.25, -25],
    [0.0, -25],
    [6.25, -25],
    [12.5, -25],
    [18.75, -25],
    [25.0, -25],
    [31.25, -25],
    [37.5, -25],
    [43.75, -25],
    [50.0, -25],
    [56.25, -25],
    [62.5, -25],
    [68.75, -25],
    [75.0, -25],
    [81.25, -25],
    [87.5, -25],
    [93.75, -25],
    [100.0, -25],
    [106.25, -25],
    [112.5, -25],
    [118.75, -25],
    [125.0, -25],
    [131.25, -25],
    [137.5

In [31]:
import calendar

In [33]:
calendar.datetime

<module 'datetime' from 'C:\\Users\\PDudukin\\Anaconda3\\lib\\datetime.py'>

In [74]:
from datetime import date

date1 = "2013-04-26 09:30:00"
date2 = "2013-04-30 09:30:00"

def diff_days(date1, date2):
    f_date = date(int(date1[:4]), int(date1[5:7]), int(date1[8:10]))
    l_date = date(int(date2[:4]), int(date2[5:7]), int(date2[8:10]))
    delta = l_date - f_date 
    return delta.days

In [84]:
event.head()

,begin,city,id,name,place,status
0,2013-04-03 12:00:00,Каменец-Подольский,61435,"Деловая игра ""Железный предприниматель"" Камене...",Место и адрес проведения уточняются.,ok
1,2013-04-06 15:00:00,Москва,61436,"Лекция Елизаветы Муратовой «Фотокомпозиция, цв...","ул. Большая Полянка, 65/74, стр. 1. FAQcafe",ok
2,2013-04-13 12:00:00,Полтава,61437,"Деловая игра ""Железный предприниматель"" Полтав...",Место и адрес проведения уточняются.,ok
3,2013-03-30 00:00:00,Без города,61438,"Семинар ""История успеха"" по технологиям Р. Кий...",NaN,ok
4,2013-03-29 12:00:00,Москва,61439,Встреча с пайщиками,"ул.Сущевская, д.21, подъезд 2, 3 этаж, офис 2,...",ok


In [86]:
event_test=event.dropna()
event_test['year']=event_test['begin'].map(lambda x: int(x[:4]))
event_test.head()

C:\Users\PDudukin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,begin,city,id,name,place,status,year
0,2013-04-03 12:00:00,Каменец-Подольский,61435,"Деловая игра ""Железный предприниматель"" Камене...",Место и адрес проведения уточняются.,ok,2013
1,2013-04-06 15:00:00,Москва,61436,"Лекция Елизаветы Муратовой «Фотокомпозиция, цв...","ул. Большая Полянка, 65/74, стр. 1. FAQcafe",ok,2013
2,2013-04-13 12:00:00,Полтава,61437,"Деловая игра ""Железный предприниматель"" Полтав...",Место и адрес проведения уточняются.,ok,2013
4,2013-03-29 12:00:00,Москва,61439,Встреча с пайщиками,"ул.Сущевская, д.21, подъезд 2, 3 этаж, офис 2,...",ok,2013
5,2013-04-03 12:00:00,Москва,61440,Встреча с пайщиками,"ул.Сущевская, д.21, подъезд 2, 3 этаж, офис 2,...",ok,2013


In [93]:
sort_event = event_test[event_test.year>2014].sort_values(by=['begin'])

In [94]:
len(sort_event)

345694

In [95]:
sort_event.head()

,begin,city,id,name,place,status,year
224667,2015-01-01 00:00:00,Киев,174854,"Ведущий,шоумен - Николай Морозов","г.Киев,ул.Ларисы Руденко,6-а\n5 этаж,офис 517",inactive,2015
222123,2015-01-01 00:00:00,Тула,172310,Клуб руководителей 2015,"г. Тула, пр-кт Ленина, 127а. Тренинговый центр...",ok,2015
220693,2015-01-01 00:00:00,Тула,170880,Программа вечерних мини-тренингов по переговор...,"г. Тула, пр-кт Ленина, 127а. Тренинговый центр...",ok,2015
239536,2015-01-01 00:00:00,Барнаул,189723,Услуги РЦИ,"Комсомольский, 118",ok,2015
220605,2015-01-01 00:00:00,Тула,170792,Программа вечерних мини-тренингов для руководи...,"г. Тула, пр-кт Ленина, 127а. Тренинговый центр...",ok,2015


In [89]:
diff_days("1970-01-01 00:00:00", sort_dates[0])

16436

In [76]:
diff_days(date1, date2)

4

In [49]:
from datetime import datetime
inDate = "29-Apr-2013-15:59:02"
d = datetime.strptime(inDate, "%d-%b-%Y-%H:%M:%S")
print(d)
datetime(2013, 4, 29, 15, 59, 2)
d.strftime("YYYYMMDD HH:mm:ss (%Y%m%d %H:%M:%S)")
print(d)

2013-04-29 15:59:02
2013-04-29 15:59:02


In [57]:
for i in range(event.shape[0]):

    np.min(event.begin.values[i])

TypeError: unorderable types: str() <= float()

### Events in dynamics on the world map by cities/categories

In [272]:
event = pd.read_csv('D:/event.csv', usecols = ['city', 'begin', 'id', 'name', 'place', 'status',])
event_coords = pd.read_csv('event_coo.csv')
event_all = pd.merge(event, event_coords, how='left', left_on='id', right_on='event_id')
e_event_category = pd.merge(event_all, event_category, how='left', left_on='id', right_on='event')
data = pd.merge(e_event_category, category, how='left', left_on='category', right_on='id')

In [273]:
data.head()

,begin,city,id_x,name_x,place,status_x,event_id,place_lat,place_lon,subw_lat,...,event,id_y,moderated,org,description,id,name_y,status_y,tag,type
0,2013-04-03 12:00:00,Каменец-Подольский,61435,"Деловая игра ""Железный предприниматель"" Камене...",Место и адрес проведения уточняются.,ok,NaN,NaN,NaN,NaN,...,61435.0,92010.0,NaN,55.0,NaN,427.0,Тренинг,inactive,NaN,event_type
1,2013-04-03 12:00:00,Каменец-Подольский,61435,"Деловая игра ""Железный предприниматель"" Камене...",Место и адрес проведения уточняются.,ok,NaN,NaN,NaN,NaN,...,61435.0,92011.0,NaN,55.0,NaN,217.0,Бизнес,ok,business,defaultcategory
2,2013-04-03 12:00:00,Каменец-Подольский,61435,"Деловая игра ""Железный предприниматель"" Камене...",Место и адрес проведения уточняются.,ok,NaN,NaN,NaN,NaN,...,61435.0,92012.0,NaN,55.0,NaN,452.0,ИТ и интернет,ok,it,defaultcategory
3,2013-04-03 12:00:00,Каменец-Подольский,61435,"Деловая игра ""Железный предприниматель"" Камене...",Место и адрес проведения уточняются.,ok,NaN,NaN,NaN,NaN,...,61435.0,92013.0,NaN,55.0,NaN,217.0,Бизнес,ok,business,defaultcategory
4,2013-04-06 15:00:00,Москва,61436,"Лекция Елизаветы Муратовой «Фотокомпозиция, цв...","ул. Большая Полянка, 65/74, стр. 1. FAQcafe",ok,NaN,NaN,NaN,NaN,...,61436.0,92008.0,NaN,14098.0,NaN,434.0,Лекция,inactive,NaN,event_type


In [274]:
data = data[(data.status_x == 'ok')]
data = data[(data.status_y == 'ok')]
data = data[(data.type == 'defaultcategory')]

In [275]:
data.columns

Index(['begin', 'city', 'id_x', 'name_x', 'place', 'status_x', 'event_id',
       'place_lat', 'place_lon', 'subw_lat', 'subw_lon', 'category', 'event',
       'id_y', 'moderated', 'org', 'description', 'id', 'name_y', 'status_y',
       'tag', 'type'],
      dtype='object')

In [276]:
data = data.dropna(subset=['id_x','place_lat','place_lon','city'], how='any')

In [277]:
geometry = [Point(xy) for xy in zip(data.place_lon, data.place_lat)]
data = GeoDataFrame(data, geometry=geometry)

In [279]:
data.columns

Index(['begin', 'city', 'id_x', 'name_x', 'place', 'status_x', 'event_id',
       'place_lat', 'place_lon', 'subw_lat', 'subw_lon', 'category', 'event',
       'id_y', 'moderated', 'org', 'description', 'id', 'name_y', 'status_y',
       'tag', 'type', 'geometry'],
      dtype='object')

In [281]:
used_cols = ['begin', 'city', 'id_x', 'name_x', 'place', 'name_y', 'geometry']

In [282]:
data = data[used_cols]
data.rename(columns={'id_x':'id', 'name_x':'name', 'name_y':'category'}, inplace=True)

C:\Users\PDudukin\Anaconda3\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [285]:
data = data.set_index('id')

In [287]:
data.shape

(326934, 6)

In [427]:
#def datetime_to_ms(datetime_str):
#    init_datetime = "1970-01-01 00:00:00"
#    
#    init_date = date(int(init_datetime[:4]), int(init_datetime[5:7]), int(init_datetime[8:10])) 
#    cur_date = date(int(datetime_str[:4]), int(datetime_str[5:7]), int(datetime_str[8:10]))
#    cur_time_ms = (int(datetime_str[11:13])*3600+int(datetime_str[14:16])*60+int(datetime_str[17:19]))*1000
#    
#    delta_dates = cur_date - init_date 
#    return [delta_dates.days*86400000+cur_time_ms]

In [428]:
def datetime_to_ms(current_datetime):
    datetime_format = "%Y-%m-%d %H:%M:%S"
    init_datetime = datetime.strptime("1970-01-01 00:00:00", datetime_format)
    curr_datetime = datetime.strptime(current_datetime, datetime_format)
    delta = curr_datetime - init_datetime
    return int(delta.total_seconds())*1000

In [ ]:
def generate_json(json):
    d = {}
    d_coords = {}
    d_times = {}
    crds_list = []
    times_list = []
    
    for idx in range(len(json)):
        crds_list.append(json[idx]['geometry']['coordinates'])
        times_list.append(json[idx]['properties']['times'][0])
    
    d_times['times'] = times_list
    d_coords['coordinates'] = crds_list
    d_coords['type'] = 'MultiPoint'
    d['geometry'] = d_coords
    d['properties'] = d_times
    d['type'] = 'Feature'
    return d

In [430]:
datetime_to_ms("2015-07-01 00:00:00")

1435708800000

In [432]:
data['times'] = data['begin'].map(lambda x: datetime_to_ms(x))
data['begin'] = data['begin'].map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
#data['year'] = data['begin'].map(lambda x: int(x[:4]))
#json_event_all[0]['properties']['times'] = [1437148800000]

TypeError: strptime() argument 1 must be str, not datetime.datetime

In [302]:
data.head()

,begin,city,name,place,category,geometry,times,year
id,,,,,,,,
61456,2013-03-27 19:00:00,Москва,К 400-летию Дома Романовых: документальное кин...,"ул.Каретный ряд, д.5/10",Кино,POINT (37.60841751098633 55.77267837524414),[1364410800000],2013
61468,2013-03-30 07:00:00,Звенигород,"""Добрая Москва""","Московская область, Одинцовский р-н, г. Звениг...",Психология и самопознание,POINT (36.85532379150391 55.72968673706055),[1364626800000],2013
61471,2013-04-19 19:00:00,Санкт-Петербург,"Открытая встреча с Александром Громовым ""Экстр...","Казанская ул, д. 7, ауд 3, \nориентир и вход -...",Психология и самопознание,POINT (30.32224655151367 59.93390274047852),[1366398000000],2013
61475,2013-04-20 11:00:00,Санкт-Петербург,Александр Громовой. Обучающие семинары по экст...,уточняется,Психология и самопознание,POINT (30.31586837768555 59.93909454345703),[1366455600000],2013
61483,2013-04-06 16:00:00,Санкт-Петербург,ПРАКТИКУМ по практической гештальт терапии,Троицкий проспект,Психология и самопознание,POINT (30.30246543884277 59.91593551635742),[1365264000000],2013


In [437]:
chosen_city = 'Москва'
chosen_category = 'Кино'
# Datetime format: YYYY-MM-DD hh:mm:ss
begin_date = datetime(2017, 1, 1, 0, 0, 0) #"2017-01-01 00:00:00"
end_date = datetime(2017, 10, 31, 0, 0, 0) #"2017-31-10 00:00:00"

cls = ['city','name','place','category','geometry','times']
json_event_all = json.loads(data[(data.city == chosen_city) & \
                                 (data.category == chosen_category) & \
                                 (data.begin >= begin_date) & \
                                 (data.begin <= end_date)][cls].sort_values(by=['times'], ascending=True).to_json())['features']
json_event_all[0]

{'geometry': {'coordinates': [37.63294982910156, 55.761451721191406],
  'type': 'Point'},
 'id': '404694',
 'properties': {'category': 'Кино',
  'city': 'Москва',
  'name': 'Ночь [аниме] кино',
  'place': 'Ул.Мясницкая 13, стр.24.',
  'times': [1483313400000]},
 'type': 'Feature'}

In [455]:
def draw_animated_events(json_file, start_date, end_date, city, category, frequency):

    cls = ['city','name','place','category','geometry','times']
    json_file = json.loads(data[(data.city == city) & \
                                     (data.category == category) & \
                                     (data.begin >= begin_date) & \
                                     (data.begin <= end_date)][cls].sort_values(by=['times'], ascending=True).to_json())['features']
    
    data_to_draw = generate_json(json_file)
    m = folium.Map([55.764414, 37.647859], zoom_start=10)
    marker_cluster = MarkerCluster().add_to(m)
    tgj = TimestampedGeoJson(data_to_draw, 
                             period=frequency, 
                             auto_play=False, 
                             loop=False, 
                             transition_time = 20,
                             #time_slider_drag_update=True
                             #loop_button=True, 
                             #min_speed = 10, 
                             #max_speed = 10
                            )
    m.add_child(tgj)
    #m
    m.save('EiT_'+city+'_'+category+'.html')

In [460]:
chosen_city = 'Москва'
chosen_category = 'Еда'
# Datetime format: YYYY-MM-DD hh:mm:ss
begin_date = datetime(2015, 1, 1, 0, 0, 0) #"2017-01-01 00:00:00"
end_date = datetime(2017, 7, 1, 0, 0, 0) #"2017-31-10 00:00:00"

draw_animated_events(json_event_all, begin_date, end_date, chosen_city, chosen_category, 'P1M')

In [453]:
set(data.category)

{'Бизнес',
 'Вечеринки',
 'Выставки',
 'Гражданские проекты',
 'Для детей',
 'Другие развлечения',
 'Другие события',
 'Еда',
 'ИТ и интернет',
 'Иностранные языки',
 'Искусство и культура',
 'Кино',
 'Концерты',
 'Красота и здоровье',
 'Образование за рубежом',
 'Психология и самопознание',
 'Спорт',
 'Театры',
 'Хобби и творчество',
 'Экскурсии и путешествия'}

### Heatmap /in Time/ + FeatureGroup

In [471]:
event = pd.read_csv('D:/event.csv', usecols = ['city', 'begin', 'id', 'name', 'place', 'status',])
event_coords = pd.read_csv('event_coo.csv')
event_all = pd.merge(event, event_coords, how='left', left_on='id', right_on='event_id')
e_event_category = pd.merge(event_all, event_category, how='left', left_on='id', right_on='event')
data = pd.merge(e_event_category, category, how='left', left_on='category', right_on='id')

In [472]:
data = data[(data.status_x == 'ok')]
data = data[(data.status_y == 'ok')]
data = data[(data.type == 'defaultcategory')]

In [473]:
data = data.dropna(subset=['id_x','place_lat','place_lon','city'], how='any')

In [474]:
used_cols = ['begin', 'city', 'id_x', 'name_x', 'place', 'name_y', 'place_lat', 'place_lon']

In [475]:
data = data[used_cols]
data.rename(columns={'id_x':'id', 'name_x':'name', 'name_y':'category'}, inplace=True)

In [488]:
data['begin'] = data['begin'].map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [489]:
data['num'] = 1.0/10

In [495]:
chosen_city = "Москва"
chosen_category = "Кино"
begin_date = datetime(2015, 1, 1, 0, 0, 0) #"2017-01-01 00:00:00"
end_date = datetime(2017, 7, 1, 0, 0, 0) #"2017-31-10 00:00:00"

def draw_heatmap(data, city, category, begin_date, end_date):
    data_heat = data[(data.city == chosen_city) & \
                    (data.category == chosen_category) & \
                    (data.begin >= begin_date) & \
                    (data.begin <= end_date)][['place_lat','place_lon','num']].values.tolist()
    m = folium.Map([55.764414, 37.647859], zoom_start=8, tiles='stamentoner')

    folium.plugins.HeatMap(data_heat).add_to(m)

    m.save('HeatMap_'+city+'_'+category+'.html')

In [496]:
draw_heatmap(data, chosen_city, chosen_category, begin_date, end_date)